In [2]:
import pandas as pd
import os
import requests
import random

In [69]:
df = pd.read_csv("table.tab", sep="\t").dropna()

In [4]:
df = df[df["CellLine"] == "."]

In [70]:
df = df[df["PrimaryCell"] != "."]

In [6]:
df.head()

,Accession,Dataset,Tissue,CellLine,PrimaryCell,Lab,Link,DataType,Controls
0,ENCFF000CWC,/experiments/ENCSR000APK/,.,.,osteoblast,"Bradley Bernstein, Broad",https://www.encodeproject.org/files/ENCFF000CW...,signal,"/files/ENCFF000CUQ/, /files/ENCFF000CUR/, /fil..."
1,ENCFF000CVQ,/experiments/ENCSR000API/,.,.,osteoblast,"Bradley Bernstein, Broad",https://www.encodeproject.org/files/ENCFF000CV...,signal,"/files/ENCFF000CUQ/, /files/ENCFF000CUR/, /fil..."
3,ENCFF000BTM,/experiments/ENCSR000ASD/,.,.,endothelial cell of umbilical vein,"Bradley Bernstein, Broad",https://www.encodeproject.org/files/ENCFF000BT...,signal,"/files/ENCFF000BQJ/, /files/ENCFF000BQK/, /fil..."
5,ENCFF000CJN,/experiments/ENCSR000ARV/,.,.,fibroblast of dermis,"Bradley Bernstein, Broad",https://www.encodeproject.org/files/ENCFF000CJ...,signal,"/files/ENCFF000CIN/, /files/ENCFF000CIO/, /fil..."
7,ENCFF001FFS,/experiments/ENCSR000DSW/,.,.,astrocyte of the spinal cord,"John Stamatoyannopoulos, UW",https://www.encodeproject.org/files/ENCFF001FF...,raw signal,"/files/ENCFF001HJK/, /files/ENCFF001HJL/, /fil..."


In [7]:
cell_lines = df["PrimaryCell"].values.tolist()

In [8]:
cell_lines = list(set(cell_lines))

In [9]:
def get_extension(file):
#     https://www.encodeproject.org/files/ENCFF429XTA/?format=json
    link_template = "https://www.encodeproject.org{}?format=json"
    r = requests.get(link_template.format(file))
    data = r.json()
    return data["file_type"]

In [10]:
cell_lines_download_map = {}

In [12]:
for line in cell_lines:
    for index,row in df.iterrows():
        if row["PrimaryCell"] == line:
            if cell_lines_download_map.get(line, None) is None:
                a = row["Controls"].split(',')
                cell_lines_download_map[line] = list(map(lambda x: x.strip(), a))
            else:
                a = row["Controls"].split(',')
                cell_lines_download_map[line] += (list(map(lambda x: x.strip(), a)))

In [154]:
cell_lines_download_map

{'fibroblast of the aortic adventitia': ['/files/ENCFF001GYT/',
  '/files/ENCFF001GYU/',
  '/files/ENCFF001GZF/',
  '/files/ENCFF798MEL/',
  '/files/ENCFF436GDS/',
  '/files/ENCFF345HOL/',
  '/files/ENCFF972ATQ/',
  '/files/ENCFF001GYT/',
  '/files/ENCFF001GYU/',
  '/files/ENCFF001GZF/',
  '/files/ENCFF798MEL/',
  '/files/ENCFF436GDS/',
  '/files/ENCFF345HOL/',
  '/files/ENCFF972ATQ/',
  '/files/ENCFF001GYT/',
  '/files/ENCFF001GYU/',
  '/files/ENCFF001GZF/',
  '/files/ENCFF798MEL/',
  '/files/ENCFF436GDS/',
  '/files/ENCFF345HOL/',
  '/files/ENCFF972ATQ/',
  '/files/ENCFF001GYT/',
  '/files/ENCFF001GYU/',
  '/files/ENCFF001GZF/',
  '/files/ENCFF798MEL/',
  '/files/ENCFF436GDS/',
  '/files/ENCFF345HOL/',
  '/files/ENCFF972ATQ/'],
 'fibroblast of dermis': ['/files/ENCFF000CIN/',
  '/files/ENCFF000CIO/',
  '/files/ENCFF000CIP/',
  '/files/ENCFF000CIU/',
  '/files/ENCFF000CIW/',
  '/files/ENCFF090CYH/',
  '/files/ENCFF138XPE/',
  '/files/ENCFF898XQK/',
  '/files/ENCFF860HKY/',
  '/files/E

In [13]:
print(len(cell_lines_download_map))

28


In [14]:
control_path = "/mnt/scratch/shared/SG_KIRILL/new_control_no_cl"
wget_commands = []
wget_template = "wget -P {} {}"
no_controls = 3
# link_template = "https://www.encodeproject.org{}@@download/{}.bigWig"
big_wig_link = "https://www.encodeproject.org{}@@download/{}.bigWig"
bam_link = "https://www.encodeproject.org{}@@download/{}.bam"
counter = 0

In [178]:
for line, controls in cell_lines_download_map.items():
    counter = 0
    for control in controls:
        
        base = control.split('/')[2]
        if counter == 3:
            break
        if get_extension(control) == "bigWig":
            wget_commands.append(
                wget_template.format(
                    control_path,
                    big_wig_link.format(control, base)
                )
            )
            counter += 1
        if get_extension(control) == "bam":
            wget_commands.append(
                wget_template.format(
                    control_path,
                    bam_link.format(control, base)
                )
            )
            counter += 1

In [179]:
output_file = open("wget_no_cl_commands.sh", "w")
output_file.write("#!/bin/bash\n")
output_file.write('\n'.join(wget_commands))
output_file.close()

In [19]:
controls_map = {}
counter = 0

In [20]:
for line, controls in cell_lines_download_map.items():
    counter = 0
    files = []
    for control in controls:
        base = control.split('/')[2]
        if counter == 3:
            break
        if get_extension(control) == "bigWig":
            files.append("{}.wig".format(base))
            counter += 1
        if get_extension(control) == "bam":
            files.append("{}.bed".format(base))
            counter += 1
    controls_map[line] = files

In [21]:
controls_map

{'fibroblast of dermis': ['ENCFF000CIN.bed',
  'ENCFF000CIO.bed',
  'ENCFF000CIP.wig'],
 'fibroblast of the aortic adventitia': ['ENCFF001GYT.bed',
  'ENCFF001GYU.wig',
  'ENCFF798MEL.bed'],
 'cardiac muscle cell': ['ENCFF001HKS.bed',
  'ENCFF001HKW.wig',
  'ENCFF293QZV.bed'],
 'mononuclear cell': ['ENCFF000VKL.bed', 'ENCFF000VKO.wig', 'ENCFF676VDZ.bed'],
 'epithelial cell of esophagus': ['ENCFF001HMG.bed',
  'ENCFF001HMH.wig',
  'ENCFF008DOI.bed'],
 'astrocyte of the cerebellum': ['ENCFF001HIW.wig',
  'ENCFF001HIX.bed',
  'ENCFF908OBD.bed'],
 'CD14-positive monocyte': ['ENCFF000CCJ.bed',
  'ENCFF000CCL.wig',
  'ENCFF000CCM.bed'],
 'keratinocyte': ['ENCFF001HVM.bed', 'ENCFF001HVO.wig', 'ENCFF969GRO.bed'],
 'epithelial cell of proximal tubule': ['ENCFF001HWM.wig',
  'ENCFF001HWN.bed',
  'ENCFF671FKH.bed'],
 'bronchial epithelial cell': ['ENCFF001HWX.bed',
  'ENCFF001HWZ.wig',
  'ENCFF948UIP.bed'],
 'brain microvascular endothelial cell': ['ENCFF001HJU.bed',
  'ENCFF001HJY.wig',
  'ENCFF

In [25]:
commands = []
base_path = "/mnt/scratch/shared/SG_KIRILL/new_control_no_cl"

In [26]:
for line, files in controls_map.items():
    names = []
    for f in files:
        names.append(os.path.join(base_path, f))
    command = "Confounder confounder={}.wig chrom=human_chrom {}".format(line.replace(" ", "_"), " ".join(names))
    commands.append(command)
out_file = open("new_cl_conf_commads.sh", "w")
out_file.write("#!/bin/bash\n")
out_file.write("\n".join(commands))
out_file.close()

In [83]:
projector_commands = []
confounder_path = "/mnt/scratch/shared/SG_KIRILL/confounders"
sample_path = "/mnt/scratch/shared/SG_KIRILL/samples"

In [84]:
downloaded_samples_f = open("dw_samples.txt")
dw_samples = downloaded_samples_f.read().splitlines()
sample_pattern = "/mnt/scratch/shared/SG_KIRILL/samples/{}.wig"
sample_bigwig_pattern = "/mnt/scratch/shared/SG_KIRILL/samples/{}.bigWig"

In [85]:
def check(name):
    if sample_bigwig_pattern.format(name) in dw_samples: 
        return True
    return False

In [86]:
for line in list(controls_map):
    tmp_df = df[df["PrimaryCell"] == line]
    accessions = tmp_df["Accession"].values.tolist()
    accessions = list(filter(check, accessions))
    accessions = list(map(lambda x: sample_pattern.format(x), accessions))
    confounder = os.path.join(confounder_path, line.replace(" ", "_"))
    PROJECTOR_COMMAND = "Projector chrom=human_chrom confounder={} {}".format(confounder, " ".join(accessions))
    projector_commands.append(PROJECTOR_COMMAND)

In [87]:
downloaded_samples_f.close()

In [88]:
out_file = open("new_cl_projector_commands.sh", "w")
out_file.write("#!/bin/bash\n")
out_file.write("\n".join(projector_commands))
out_file.close()

In [90]:
projector_commands

['Projector chrom=human_chrom confounder=/mnt/scratch/shared/SG_KIRILL/confounders/fibroblast_of_dermis /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CJN.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CJZ.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CKS.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CKF.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CKX.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CKL.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CLD.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CJI.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CJU.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CLJ.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CLS.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF000CLJ.wig',
 'Projector chrom=human_chrom confounder=/mnt/scratch/shared/SG_KIRILL/confounders/fibroblast_of_the_aortic_adventitia /mnt/scratch/shared/SG_KIRILL/samples/ENCFF001GYR.wig /mnt/scratch/shared/SG_KIRILL/samples/ENCFF001GYS.wig /mnt/scratch/shared